### Logistic Regression
- 종속 변수가 이항 변수(binary variable)인 경우에 사용
- 종속 변수가 이항 변수(binary variable)
- 로지스틱 함수(Logistic Function)를 사용하여 확률을 예측
- 비교적 간단하고 빠르게 학습
- 이상치(outlier)에 민감
- 다중 공선성(multicollinearity)이 있는 경우, 결과가 왜곡

In [1]:
#data analytics
import pandas as pd
from scipy.stats import pearsonr

#sklearn
import sklearn as sk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [4]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/데이터_전처리_파일.csv', encoding='cp949')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 머신러닝 학습
- 상관관계가 높은 순서대로 변수 삽입
- 견적화폐(0.33), Machinery(0.17), Subject(0.13) , Part No.1, key1, Assembly...
- 주성분분석이 높은 변수 삽입
- Subject, 출고운반선, Control No., Assembly ...

In [25]:
# 텍스트 데이터
data = list( df['Subject'] + ' ' + df['출고운반선'] + ' ' + df['Control No.'] + ' ' + df['Machinery'])

# target 값
target =list(df['리드타임_음수제거'])

# CountVectorizer를 이용하여 feature 벡터 생성
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data)

# df[리드타임_음수제거]는 별도의 인코딩이 필요치않음
y = target

# train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 정확도 함수
from sklearn.metrics import accuracy_score 
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score

def print_metrics(y_test, X_test, model) :
  # 테스트 데이터셋으로 예측
  y_pred = model.predict(X_test)

  # 각종 평가지표 계산
  recall = recall_score(y_test, y_pred, average='macro')
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='macro')
  f1 = f1_score(y_test, y_pred, average='macro')

  print("Recall: {:.4f}".format(recall))
  print("Accuracy: {:.4f}".format(accuracy))
  print("Precision: {:.4f}".format(precision))
  print("F1-score: {:.4f}".format(f1))

In [26]:
from sklearn.linear_model import LogisticRegression

# Logistic Regression 모델 생성 및 학습
lr = LogisticRegression(random_state=30, C=8, solver="liblinear",  multi_class="ovr", max_iter=100)
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_acc = accuracy_score(y_test, lr_pred)
print_metrics(y_test, X_test, lr)


Recall: 0.8361
Accuracy: 0.9347
Precision: 0.8553
F1-score: 0.8405


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
# 피클로 모델 저장
import pickle 
with open('lead_LR.pickle','wb') as fw:
    pickle.dump(lr, fw)

In [28]:
# vectorizer 생성
vectorizer = CountVectorizer()

# feature 벡터 생성
X = vectorizer.fit_transform(data)

# vectorizer 객체 저장
with open('lead_LR_vectorizer.pickle', 'wb') as fw:
    pickle.dump(vectorizer, fw)